"DyLoRA: Parameter Efficient Tuning of Pre-trained Models using Dynamic Search-Free Low Rank Adaptation" presenta una innovación en la adaptación de modelos pre-entrenados, centrándose en la eficiencia de parámetros y la reducción de la necesidad de recursos. Los autores señalan que la afinación de los modelos pre-entrenados de gran tamaño es costosa y consume muchos recursos. Los autores introducen DyLoRA, una adaptación de bajo rango dinámica que aborda las limitaciones de LoRA. DyLoRA entrena bloques de LoRA para un rango de rangos, en lugar de un rango único, y clasifica la representación aprendida en diferentes rangos durante el entrenamiento.

Para ellos, los resultados muestran que DyLoRA es al menos 7 veces más rápido que LoRA en entrenamiento, sin comprometer significativamente el rendimiento. Además, DyLoRA funciona bien en un rango mucho más amplio de rangos en comparación con LoRA, porque DyLoRA aborda eficazmente dos problemas en los adaptadores de bajo rango: la selección de rango y la dinámica en tiempo de inferencia, logrando evitar la búsqueda de rangos óptimos en escenarios de la vida real con un rendimiento comparable.

DyLoRA no aplica SVD, pero, LoRA si. De hecho, al "congelar" los demás datos y construir bloques usando SVD, LoRA hace que sean eficientes en términos de parámetros, pero tienen dos problemas principales:

Tamaño Fijo de los Bloques: El tamaño de estos bloques es fijo y no se puede modificar después del entrenamiento. Por ejemplo, si necesitamos cambiar el rango de los bloques LoRA, tendríamos que reentrenarlos desde cero.

Optimización del Rango: Optimizar el rango de estos bloques requiere una búsqueda exhaustiva y un esfuerzo considerable.

DyLoRA, o Adaptación de Bajo Rango Dinámica, aborda estos dos problemas. A diferencia de LoRA, que utiliza una versión truncada y fija de SVD, DyLoRA introduce una forma dinámica de adaptación de bajo rango. Esto significa que en lugar de tener bloques con un tamaño y rango fijos, DyLoRA permite que estos aspectos sean flexibles y se ajusten dinámicamente. Esto puede facilitar la adaptación del modelo a diferentes tareas o datos sin necesidad de reentrenar desde cero y simplifica el proceso de encontrar el rango óptimo para los bloques LoRA.

Para utilizar DyLoRA solo hemos adaptado la clase y la parametrización de los pesos.



https://neurips2022-enlsp.github.io/papers/paper_37.pdf

In [ ]:
#importamos las librerías necesarias
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# hacemos torch determinístico
_ = torch.manual_seed(0)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# cargamos el dataset MNIST para train
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# creamos un dataloader para train
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# cargamos MNIST para test
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

# definimos el device que vamos utilizar
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# creamos una red neuronal demasiado ineficiente para clasificar dígitos MNIST

class desperdicio(nn.Module):
    def __init__(self, hidden_size_1=1000, hidden_size_2=2000):
        super(desperdicio,self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

net = desperdicio().to(device)

In [ ]:
def train(train_loader, net, epochs=5, total_iterations_limit=None):
    cross_el = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    total_iterations = 0

    for epoch in range(epochs):
        net.train()

        loss_sum = 0
        num_iterations = 0

        data_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:
            num_iterations += 1
            total_iterations += 1
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = net(x.view(-1, 28*28))
            loss = cross_el(output, y)
            loss_sum += loss.item()
            avg_loss = loss_sum / num_iterations
            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return

train(train_loader, net, epochs=1)

Epoch 1: 100%|██████████| 6000/6000 [08:11<00:00, 12.22it/s, loss=0.236]


In [ ]:
#salvamos los pesos originales para poder recuperarlos
original_weights = {}
for name, param in net.named_parameters():
    original_weights[name] = param.clone().detach()

In [ ]:
#hacemos test en las clasificaciones donde vemos que se equivoca mucho en el dígito 9 en relación a los demás
def test():
    correct = 0
    total = 0

    wrong_counts = [0 for i in range(10)]

    with torch.no_grad():
        for data in tqdm(test_loader, desc='Testing'):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = net(x.view(-1, 784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct +=1
                else:
                    wrong_counts[y[idx]] +=1
                total +=1
    print(f'Accuracy: {round(correct/total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the digit {i}: {wrong_counts[i]}')

test()

Testing: 100%|██████████| 1000/1000 [00:06<00:00, 164.47it/s]

Accuracy: 0.954
wrong counts for the digit 0: 31
wrong counts for the digit 1: 17
wrong counts for the digit 2: 46
wrong counts for the digit 3: 74
wrong counts for the digit 4: 29
wrong counts for the digit 5: 7
wrong counts for the digit 6: 36
wrong counts for the digit 7: 80
wrong counts for the digit 8: 25
wrong counts for the digit 9: 116


In [ ]:
# salvamos los parámetros originales
total_parameters_original = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    total_parameters_original += layer.weight.nelement() + layer.bias.nelement()
    print(f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape}')
print(f'Total number of parameters: {total_parameters_original:,}')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10])
Total number of parameters: 2,807,010


DyLoRA permite ajustar dinámicamente tanto el rango (r) como el valor alfa (α), lo que significa que estos parámetros pueden cambiar durante el entrenamiento para adaptarse mejor a la tarea en curso. En contraste, LoRA fija estos valores y no los ajusta durante el entrenamiento.

En la implementación de DyLoRA, la función adjust_rank_and_alpha permite cambiar el rango (r) y el valor alfa (α) de manera dinámica y reinicializar los parámetros correspondientes. Esto significa que se puede experimentar con diferentes valores de rango y alfa durante el entrenamiento para encontrar la configuración óptima.

In [ ]:
#aquí hacemos el cambio de LoRA a DyLoRA
class DynamicLoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out, rank=1, alpha=1, device='cpu'):
        super(DynamicLoRAParametrization, self).__init__()
        self.rank = rank
        self.alpha = alpha
        self.features_in = features_in
        self.features_out = features_out
        self.device = device
        self.lora_A = nn.Parameter(torch.zeros((rank, features_out)).to(device))
        self.lora_B = nn.Parameter(torch.zeros((features_in, rank)).to(device))
        nn.init.normal_(self.lora_A, mean=0, std=1)
        self.scale = self.alpha / self.rank

    def forward(self, original_weights):
        # Return W + scaled(BA)
        delta_W = torch.matmul(self.lora_B, self.lora_A) * self.scale
        return original_weights + delta_W.view(original_weights.shape)

    def adjust_rank_and_alpha(self, new_rank, new_alpha):
        # Dynamically adjust `rank` and `alpha`, and reinitialize parameters
        self.rank = new_rank
        self.alpha = new_alpha
        self.lora_A = nn.Parameter(torch.zeros((self.rank, self.features_out)).to(self.device))
        self.lora_B = nn.Parameter(torch.zeros((self.features_in, self.rank)).to(self.device))
        nn.init.normal_(self.lora_A, mean=0, std=1)
        self.scale = self.alpha / self.rank

In [ ]:
import torch.nn.utils.parametrize as parametrize


def linear_layer_parameterization_dynamic(layer, device, rank=1, lora_alpha=1):
    # solo añadimos la parametrización a la matriz de pesos, ignoramos el sesgo

    # Utilizamos la versión dinámica de LoRAParametrization
    features_in, features_out = layer.weight.shape
    return DynamicLoRAParametrization(
        features_in, features_out, rank=rank, alpha=lora_alpha, device=device
    )

# Registramos la parametrización dinámica para las capas
parametrize.register_parametrization(
    net.linear1, "weight", linear_layer_parameterization_dynamic(net.linear1, device)
)
parametrize.register_parametrization(
    net.linear2, "weight", linear_layer_parameterization_dynamic(net.linear2, device)
)
parametrize.register_parametrization(
    net.linear3, "weight", linear_layer_parameterization_dynamic(net.linear3, device)
)

def enable_disable_lora_dynamic(enabled=True):
    for layer in [net.linear1, net.linear2, net.linear3]:
        layer.parametrizations.weight[0].enabled = enabled

In [ ]:
# Simplemente calculamos el total en este contexto
total_parameters = sum(p.numel() for p in net.parameters())
print(f'Total number of parameters (including LoRA): {total_parameters:,}')



Total number of parameters (including LoRA): 2,813,804


In [ ]:
# congelamos los parámetros no-Lora
for name, param in net.named_parameters():
    if 'lora' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

# cargamos el mnist otra vez pero solo con el 9 (fine-tuning)
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
exclude_indices = mnist_trainset.targets == 9
mnist_trainset.data = mnist_trainset.data[exclude_indices]
mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# creamos un dataloader para el training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# entrenamos la red con DyLoRA solo en el dígito 9 y solo en 100 batches
train(train_loader, net, epochs=1, total_iterations_limit=100)

Freezing non-LoRA parameter linear1.bias
Freezing non-LoRA parameter linear1.parametrizations.weight.original
Freezing non-LoRA parameter linear2.bias
Freezing non-LoRA parameter linear2.parametrizations.weight.original
Freezing non-LoRA parameter linear3.bias
Freezing non-LoRA parameter linear3.parametrizations.weight.original


Epoch 1:  99%|█████████▉| 99/100 [00:04<00:00, 21.27it/s, loss=0.0265]


In [ ]:

full_test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=False, download=True, transform=transform),
    batch_size=10,
    shuffle=False
)


correct_predictions_9 = 0
total_samples_9 = 0

with torch.no_grad():
    for images, labels in full_test_loader:

        digit_9_indices = labels == 9
        images_9 = images[digit_9_indices]
        labels_9 = labels[digit_9_indices]

        if len(images_9) > 0:
            outputs = net(images_9)  # pasamos nuestro modelo tuneado
            _, predicted = torch.max(outputs, 1)
            correct_predictions_9 += (predicted == labels_9).sum().item()
            total_samples_9 += labels_9.size(0)


accuracy_9 = (correct_predictions_9 / total_samples_9) * 100
print(f'Accuracy on digit 9 using fine-tuned model: {accuracy_9:.2f}%')


Accuracy on digit 9 using fine-tuned model: 99.90%
